In [1]:
import os

import pandas as pd

from cobra.io import read_sbml_model
from cobra.flux_analysis import pfba

In [2]:
output_dir = './analysis_output'
model_output_dir = './model_files'

if output_dir[-1] == '/':
    output_dir = output_dir[:-1]

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [4]:
def calculate_flux(model, target_chem, c_source='glc__D_e'):
    target_chem_name = model.metabolites.get_by_id(target_chem).name
    c_source_name = model.metabolites.get_by_id(c_source).name
    target_rxn_id = 'EX_' + target_chem
    c_source_id = 'EX_' + c_source
    with model as m:
        m.reactions.ATPM.knock_out()
        m.objective = target_rxn_id
        sol = pfba(m)
    f_target = sol.fluxes[target_rxn_id]
    f_c = -sol.fluxes[c_source_id]
    yt_mol = f_target/f_c

    print(f'{yt_mol: 0.6f} {target_chem_name} mol/ {c_source_name} mol')
    return yt_mol, f_target, f_c

In [5]:
model_bz_dp = read_sbml_model(f'{model_output_dir}/iML1515_bz_dp.xml')
model_bz_idp = read_sbml_model(f'{model_output_dir}/iML1515_bz_idp.xml')

Academic license - for non-commercial use only - expires 2024-06-11
Using license file /data/user_home/gbkim/gurobi.lic


### Flux calculation using integrated strains

In [7]:
flux_data = []

target_chem = 'bzac_e'
target_chem_name = 'benzyl acetate'

anal_desc = [
    'Integrated strain-benzoic acid dependent pathway',
    'Integrated strain-benzoic acid independent pathway', 
]

anal_models = [model_bz_dp, model_bz_idp]

for desc, item in zip(anal_desc, anal_models):
    y, f, fc = calculate_flux(item, target_chem, c_source='glc__D_e')
    data = [desc, target_chem_name, f, fc, y]
    flux_data.append(data)
    
    
df_integrated = pd.DataFrame(flux_data)
df_integrated.columns = [
    'Description', 'Target chemical', 
    'Target chemical production flux (mmol/gDCW/h)', 'Carbon source uptake flux (mmol/gDCW/h)',
    'molar yield (mol/mol)', 
]
df_integrated.to_csv(f'{output_dir}/integrated_strains_flux.txt', sep='\t')
df_integrated

 0.487487 Benzyl acetate mol/ D-Glucose mol
 0.436735 Benzyl acetate mol/ D-Glucose mol


,Description,Target chemical,Target chemical production flux (mmol/gDCW/h),Carbon source uptake flux (mmol/gDCW/h),molar yield (mol/mol)
0,Integrated strain-benzoic acid dependent pathway,benzyl acetate,4.874875,10.0,0.487487
1,Integrated strain-benzoic acid independent pat...,benzyl acetate,4.367347,10.0,0.436735
